In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import numpy as np
import os
import logging
from data_load import EnhancedWildScenesDataset
from models.unet import UNet
from utils.metrics import calculate_miou_train, calculate_pixel_accuracy, calculate_dice_coefficient
from utils.losses import CombinedLoss
from utils.log import setup_logger, save_checkpoint
import torch.nn.functional as F

def setup_logger(log_file):
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        handlers=[
                            logging.FileHandler(log_file),
                            logging.StreamHandler()
                        ])

def save_checkpoint(model, optimizer, epoch, metrics, path):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': metrics
    }
    torch.save(state, path)

def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, num_classes, scaler):
    model.train()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0
    num_batches = 0

    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:], mode='bilinear', align_corners=False)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pred = torch.argmax(outputs, dim=1)
        miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
        pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
        dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

        if not np.isnan(miou):
            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice
            num_batches += 1

    return (total_loss / len(dataloader),
            total_miou / num_batches if num_batches > 0 else 0.0,
            total_pixel_acc / num_batches if num_batches > 0 else 0.0,
            total_dice / num_batches if num_batches > 0 else 0.0)

def validate_epoch(model, dataloader, criterion, device, num_classes):
    model.eval()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:], mode='bilinear', align_corners=False)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            pred = torch.argmax(outputs, dim=1)
            miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
            pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
            dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice

    num_batches = len(dataloader)
    return (total_loss / num_batches,
            total_miou / num_batches,
            total_pixel_acc / num_batches,
            total_dice / num_batches)

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir):
    best_loss = float('inf')
    best_miou = 0
    scaler = GradScaler()

    for epoch in range(num_epochs):
        logging.info(f"Epoch {epoch + 1}/{num_epochs}")

        train_loss, train_miou, train_pixel_acc, train_dice = train_epoch(model, train_loader, criterion, optimizer, scheduler, device, num_classes, scaler)
        logging.info(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Train mIoU: {train_miou:.4f}, "
                     f"Train Pixel Acc: {train_pixel_acc:.4f}, Train Dice: {train_dice:.4f}")

        val_loss, val_miou, val_pixel_acc, val_dice = validate_epoch(model, val_loader, criterion, device, num_classes)
        logging.info(f"Epoch {epoch + 1} - Val Loss: {val_loss:.4f}, Val mIoU: {val_miou:.4f}, "
                     f"Val Pixel Acc: {val_pixel_acc:.4f}, Val Dice: {val_dice:.4f}")

        scheduler.step(val_loss)

        metrics = {
            'val_loss': val_loss,
            'val_miou': val_miou,
            'val_pixel_acc': val_pixel_acc,
            'val_dice': val_dice
        }

        if val_miou > best_miou:
            best_miou = val_miou
            best_model_path = os.path.join(save_dir, f'best_model_epoch_{epoch + 1}.pth')
            save_checkpoint(model, optimizer, epoch + 1, metrics, best_model_path)
            logging.info(f"Epoch {epoch + 1} - Best model saved with Val mIoU: {best_miou:.4f}")

        if (epoch + 1) % 5 == 0:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{epoch + 1}.pth')
            save_checkpoint(model, optimizer, epoch + 1, metrics, checkpoint_path)
            logging.info(f"Epoch {epoch + 1} - Checkpoint saved")

        current_lr = optimizer.param_groups[0]['lr']
        logging.info(f"Current learning rate: {current_lr:.6f}")

    logging.info(f"Training completed after {num_epochs} epochs.")
    return best_model_path

if __name__ == "__main__":
    save_dir = 'model_checkpoints/Unet'
    os.makedirs(save_dir, exist_ok=True)

    log_file = os.path.join(save_dir, 'training.log')
    setup_logger(log_file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")

    train_loader = EnhancedWildScenesDataset.get_data_loader('train', batch_size=8)
    val_loader = EnhancedWildScenesDataset.get_data_loader('valid', batch_size=8)

    num_classes = 18

    model = UNet(3, num_classes).to(device)

    criterion = CombinedLoss(weight_focal=1.0, weight_dice=0.5)

    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5, weight_decay=0.01)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    num_epochs = 30
    best_model_path = train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir)
    logging.info(f"Training completed! Best model saved at {best_model_path}")


2024-07-18 23:00:53,688 - INFO - Using device: cuda
2024-07-18 23:00:53,959 - INFO - Epoch 1/30
Training: 100%|██████████████████████████████████████████████████████████████████████| 137/137 [08:05<00:00,  3.54s/it]
2024-07-18 23:08:59,236 - INFO - Epoch 1 - Train Loss: 2.1668, Train mIoU: 0.1777, Train Pixel Acc: 0.5775, Train Dice: 0.2115
Validating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [02:16<00:00,  3.50s/it]
2024-07-18 23:11:15,574 - INFO - Epoch 1 - Val Loss: 1.6074, Val mIoU: 0.4863, Val Pixel Acc: 0.7662, Val Dice: 0.5237
2024-07-18 23:11:15,912 - INFO - Epoch 1 - Best model saved with Val mIoU: 0.4863
2024-07-18 23:11:15,912 - INFO - Current learning rate: 0.000050
2024-07-18 23:11:15,913 - INFO - Epoch 2/30
Training: 100%|██████████████████████████████████████████████████████████████████████| 137/137 [07:55<00:00,  3.47s/it]
2024-07-18 23:19:11,514 - INFO - Epoch 2 - Train Loss: 1.4664, Train mIoU: 0.4233, Train Pixel Acc: 0.7909,

In [8]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import os
import logging
from data_load import EnhancedWildScenesDataset
from models.unet import UNet
from utils.metrics import calculate_miou_train, calculate_pixel_accuracy, calculate_dice_coefficient
import matplotlib.pyplot as plt

def setup_logger(log_file):
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        handlers=[
                            logging.FileHandler(log_file),
                            logging.StreamHandler()
                        ])

def test(model, dataloader, device, num_classes, save_dir):
    model.eval()
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0
    num_batches = 0
    image_count = 0

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(dataloader, desc="Testing")):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:], mode='bilinear', align_corners=False)

            pred = torch.argmax(outputs, dim=1)
            miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
            pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
            dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

            if not np.isnan(miou):
                total_miou += miou
                total_pixel_acc += pixel_acc
                total_dice += dice
                num_batches += 1

            # Save ground truth and prediction for all images in the batch
            for j in range(images.shape[0]):
                save_comparison(images[j], labels[j], pred[j], image_count, save_dir)
                image_count += 1

    logging.info(f"Total images processed and saved: {image_count}")

    return (total_miou / num_batches if num_batches > 0 else 0.0,
            total_pixel_acc / num_batches if num_batches > 0 else 0.0,
            total_dice / num_batches if num_batches > 0 else 0.0)
def normalize_image(image):
    """Normalize image tensor to [0, 1] range."""
    image = image.cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())
    return np.clip(image, 0, 1)

def save_comparison(image, ground_truth, prediction, index, save_dir):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Normalize and convert image to correct format
    image_np = normalize_image(image)
    image_np = np.transpose(image_np, (1, 2, 0))  # Change from (C, H, W) to (H, W, C)
    
    # Original image
    ax1.imshow(image_np)
    ax1.set_title("Original Image")
    ax1.axis('off')
    
    # Ground truth
    ax2.imshow(ground_truth.cpu().numpy(), cmap='viridis')
    ax2.set_title("Ground Truth")
    ax2.axis('off')
    
    # Prediction
    ax3.imshow(prediction.cpu().numpy(), cmap='viridis')
    ax3.set_title("Prediction")
    ax3.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'comparison_{index}.png'))
    plt.close()

if __name__ == "__main__":
    save_dir = 'prediction/Unet'
    log_file = os.path.join(save_dir, 'testing.log')
    
    os.makedirs(save_dir, exist_ok=True)
    
    setup_logger(log_file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")

    test_loader = EnhancedWildScenesDataset.get_data_loader('test', batch_size=8)

    num_classes = 18

    model = UNet(3, num_classes).to(device)
    
    # 使用相對路徑加載最佳模型，並添加更多可能的路徑
    best_model_path = os.path.join('model_checkpoints', 'Unet', 'best_model_epoch_25.pth')
    
    model_loaded = False
    if os.path.exists(best_model_path):
        checkpoint = torch.load(best_model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        logging.info(f"Loaded model from {path}")
        model_loaded = True
    
    if not model_loaded:
        logging.error(f"Model file not found. Tried paths: {', '.join(possible_model_paths)}")
        exit(1)

    miou, pixel_acc, dice = test(model, test_loader, device, num_classes, save_dir)
    
    logging.info(f"Test Results:")
    logging.info(f"Mean IoU: {miou:.4f}")
    logging.info(f"Pixel Accuracy: {pixel_acc:.4f}")
    logging.info(f"Dice Coefficient: {dice:.4f}")

2024-07-19 04:45:22,943 - INFO - Using device: cuda
2024-07-19 04:45:23,635 - INFO - Loaded model from model_checkpoints\Unet\best_model_epoch_25.pth
Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [01:44<00:00,  5.50s/it]
2024-07-19 04:47:08,155 - INFO - Total images processed and saved: 152
2024-07-19 04:47:08,157 - INFO - Test Results:
2024-07-19 04:47:08,158 - INFO - Mean IoU: 0.6852
2024-07-19 04:47:08,158 - INFO - Pixel Accuracy: 0.8657
2024-07-19 04:47:08,159 - INFO - Dice Coefficient: 0.7246
